In [1]:
# Loading in functions
import os
import csv
import torch
import pandas as pd
from os import listdir
from model import DogDetectorModel
from image_to_tensor import to_tensor
from non_max_surpression import NonMaxSurpression

In [ ]:
def dog_extractor(image):

    # Initializing the images path
    img_path = "/Users/karthik/Desktop/open-images-v6/train/data/" + image

    # Initializing the model
    model = DogDetectorModel()

    # Initializing Non Max Surpression with default parameters
    nms = NonMaxSurpression()

    # Loading the checkpoint
    checkpoint = torch.load('model.pt', map_location=torch.device('cpu')) 

    # Loading the model weights
    model.load_state_dict(checkpoint['model_state_dict'])

    # Setting model to evaluation model
    model = model.eval()

    # Loading and transforming images
    images = to_tensor([img_path])

    # Running image through model
    outputs = model(images, '')

    # Executing non max surpression on the model output
    outputs = nms(outputs)

    return outputs



vals = []
folder_dir = "/Users/karthik/Desktop/open-images-v6/train/data"
img_files = os.listdir(folder_dir)

df = pd.read_csv('processed.csv', names=['image', 'width', 'height', 'area'], header=None)
csv_vals_list = df['image']

reqd_list = list(set(img_files) - set(csv_vals_list))

for images in reqd_list:
    if (images.endswith(".png") or images.endswith(".jpg") or images.endswith(".jpeg")):

        # Output of the parameter
        output = dog_extractor(images)
        b_box = output[0]['boxes'][0]

        # Calculating features
        w = b_box[2] - b_box[0]
        h = b_box[3] - b_box[1]
        area = w * h
        features = {'width': w,
                    'height': h,
                    'area': area}
        
        fields=[images, w, h, area]

        with open(r'processed.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
        
        vals.append(features)
        print(area)

In [2]:
df = pd.read_csv('processed.csv', names=['image', 'width', 'height', 'area'], header=None)

In [3]:
del df['image']

In [4]:
df.to_csv('km.csv')

In [5]:
data = df

In [6]:
# standardizing the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# statistics of scaled data
pd.DataFrame(data_scaled).describe()

,0,1,2
count,6.010000e+02,6.010000e+02,6.010000e+02
mean,9.698288e-18,3.911643e-17,-1.089903e-17
std,1.000833e+00,1.000833e+00,1.000833e+00
min,-1.515458e+00,-1.622996e+00,-5.297689e-01
25%,-7.816083e-01,-8.016186e-01,-3.926206e-01
50%,-1.412687e-01,1.785945e-02,-1.377185e-01
75%,6.822344e-01,6.168713e-01,2.278793e-01
max,1.108197e+01,1.133917e+01,2.226231e+01


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import datasets

x = data.iloc[:, [0,1,2]]
x  = np.array(x)

# Define the model
kmeans_model = KMeans(n_clusters=3)
# Fit into our dataset fit
kmeans_predict = kmeans_model.fit_predict(x)

In [1]:
data['cluster'] = kmeans_predict

NameError: name 'kmeans_predict' is not defined

In [ ]:
data

In [ ]:
# Visualising the clusters
plt.scatter(x[kmeans_predict == 0, 0], x[kmeans_predict == 0, 1], s=100, c = 'red', label = 'Large')
plt.scatter(x[kmeans_predict == 1, 0], x[kmeans_predict == 1, 1], s=100, c = 'blue', label = 'Medium')
plt.scatter(x[kmeans_predict == 2, 0], x[kmeans_predict == 2, 1], s=100, c = 'green', label = 'Small')
# Plotting the centroids of the clusters
plt.scatter(kmeans_model.cluster_centers_[:, 0], kmeans_model.cluster_centers_[:,1], s=100, c = 'yellow', label = 'Centroids')
plt.legend()